In [2]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [3]:
files = glob('topics/*.json')
len(files)

97

In [6]:
!mkdir threads

In [7]:
forums = []
for f in files:
    with open(f) as fopen:
        forums.extend(json.load(fopen))
        
forums = sorted(list(set(forums)))
len(forums)

316928

In [10]:
forums

['/threads/0-02-9-23-when-even-god-abandons-you-part-2-3-10-year-of-terror.6861177/',
 '/threads/0-05-chicken-wings.6840088/',
 '/threads/0-05-fried-potato.6842380/',
 '/threads/0-05-ice-cream.6840798/',
 '/threads/0-05-is-attractive-interest.5605556/',
 '/threads/0-15mm-tempered-glass-screen-protector.4478210/',
 '/threads/0-20-fries.6834176/',
 '/threads/0-20-lighter.6894213/',
 '/threads/0-5-ft-15-cm-rca-male-to-male-cable.4975562/',
 '/threads/0-5.6886406/',
 '/threads/0-50-for-quail-egg-diner-flames-mala-stall-over-predatory-pricing.6877876/',
 '/threads/0-50-large-egg-roti-prata.6852859/',
 '/threads/0-5kgs-olympic-weight-plate.5833346/',
 '/threads/0-60-cents-hourly.6895130/',
 '/threads/0-70-noodles-best-cheap-eats-in-singapore.6833468/',
 '/threads/0-78mm-to-mmcx-converter.6152588/',
 '/threads/0-corbid-36-china-cities-in-some-kind-of-lockdown-almost-200m-ppl-affected.6821804/',
 '/threads/0-downpayment-100-loan-with-interest-rate-at-1-99-how-true-is-that.2889518/',
 '/threads

In [9]:
topic = forums[0]
f'https://forums.hardwarezone.com.sg{topic}'

'https://forums.hardwarezone.com.sg/threads/0-02-9-23-when-even-god-abandons-you-part-2-3-10-year-of-terror.6861177/'

In [12]:
import time

def get_href_topic(url, topic = None):
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    if topic:
        try:
            max_page = max([int(a_.split('page-')[-1]) for a_ in a if topic in a_ and '/page-' in a_])
        except:
            max_page = -1
    else:
        max_page = -1
    div = soup.find_all('div', {'class': 'bbWrapper'})
    return url, [str(d) for d in div], max_page

In [15]:
topic = '/threads/hoseh-boh-open-mob-on-beautiful-saturday.6905634/'


In [16]:
get_href_topic(f'https://forums.hardwarezone.com.sg{topic}', topic)

('https://forums.hardwarezone.com.sg/threads/hoseh-boh-open-mob-on-beautiful-saturday.6905634/',
 ['<div class="bbWrapper"><a class="link link--external" href="https://www.mindef.gov.sg/web/portal/mindef/news-and-events/latest-releases/article-detail/2023/May/20may23_nr" rel="nofollow ugc noopener" target="_blank">https://www.mindef.gov.sg/web/porta...t-releases/article-detail/2023/May/20may23_nr</a></div>',
  '<div class="bbWrapper">heng not me<br/>\n<br/>\n<img alt=":frown:" class="smilie" data-shortname=":frown:" loading="lazy" src="https://www.hardwarezone.com.sg/img/forums/hwz/smilies/frown.gif" title="Frown    :frown:"/></div>',
  '<div class="bbWrapper">MR loh <br/>\n<br/>\n10 char</div>',
  '<div class="bbWrapper"><blockquote class="bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch">\n<div class="bbCodeBlock-title">\n<a class="bbCodeBlock-sourceJump" data-content-selector="#post-147642278" data-xf-click="attribution" href="/goto/post?id=147642278" rel="nofol

In [18]:
max_worker = 30
for i in tqdm(range(0, len(forums), max_worker)):
    filename = os.path.join('threads', f'{i}.json')
    if os.path.exists(filename):
        continue
    
    gc.collect()
    
    urls = [(f'https://forums.hardwarezone.com.sg{t}', t) for t in forums[i: i + max_worker]]
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href_topic, url[0], url[1]): url for url in urls}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

 91%|████████████████████████████████▊   | 9616/10565 [4:40:43<30:49,  1.95s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████| 10565/10565 [5:03:40<00:00,  1.72s/it]
